In [1]:
%matplotlib inline

In [1]:
from sklearn import linear_model
import matplotlib.pyplot as plt
import time
import Xlib.display as display
import gtk, pygtk
import csv
import numpy as np
import ConfigParser
import tensorflow as tf

/usr/lib/python2.7/dist-packages/gtk-2.0/gtk/__init__.py:127: RuntimeWarning: PyOS_InputHook is not available for interactive use of PyGTK
  set_interactive(1)


In [2]:
theta = 3.5e-07

In [3]:
#read write project path
#TODO: configure this file according to your path: BehavioralAuthentication/src/main/php/configuration.cfg
config = ConfigParser.RawConfigParser()
config.read('configuration.cfg')

#project_dir_path = config.get('Section1', 'project_dir')
#resource_path = project_dir_path + "src/test/resources/"

[]

In [4]:
window = gtk.Window()
screen = window.get_screen()

x_size = screen.get_width()
y_size = screen.get_height()

In [7]:
#make new

#ted dunning model

def extractFeatures(filename, n_last_timestamps_as_features = 200, normalize = True, bias = True):
    queue = {}
    
    diffTime = {}
    oldTime = -1.0
    
    maxX = 0.0
    
    x_width = float(screen.get_width()) 
    y_width = float(screen.get_height()) 
    
    
    

    x_attribute = -1
    y_attribute = -1
    i = 0
    r = 0
    
    record_number = len(list(csv.reader(open(filename))))
    print record_number
    
    with open(filename, 'r') as csvfile:
         spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')   
            
         for row in spamreader:
            
            feature_num = len(row)
            
            data_size = record_number - n_last_timestamps_as_features - 1
            
            if (i==0):
                
                data = np.zeros((data_size, n_last_timestamps_as_features, feature_num))
                target = np.zeros((data_size, feature_num))
                
                #find mouse position coordinate columns in the file
                for a in range(feature_num):
                    if (row[a] == "mouse_X"):
                        x_attribute = a
                    if (row[a] == "mouse_Y"):
                        y_attribute = a
                        
            else:
                #normalize mouse coordinates
                if (normalize):
                     row[x_attribute] = float(row[x_attribute]) / x_width   
                     row[y_attribute] = float(row[y_attribute]) / y_width
                
                
                if (i <= n_last_timestamps_as_features):
                    data [0,(i-1),:] = row
                    
                elif(i == n_last_timestamps_as_features + 1):
                    target[0,:] = row
                    
                    r = r + 1
                    
                    
                else:
                    data[r, 0 : (n_last_timestamps_as_features - 1), :] = data[(r - 1), 1 : n_last_timestamps_as_features, :]
                    data[r, (n_last_timestamps_as_features - 1), :] = target[(r - 1), :]
                        
                    target[r,:] = row                   
                
                    r = r + 1
            i = i + 1
            
    #if (bias):
    #    data = np.hstack((data, np.ones((data.shape[0], 1), dtype=data.dtype)))
    
    
    print("original shape: " + str(data.shape))
    

    return ((data, target))
            
            
 

In [9]:
extractFeatures("/home/felix/mousetracking/BehavioralAuthentication/src/main/java/features4.txt", \
                n_last_timestamps_as_features = 2, normalize = False, bias = True)

9
original shape: (6, 2, 2)


(array([[[  1.,   2.],
         [  3.,   4.]],
 
        [[  3.,   4.],
         [  5.,   6.]],
 
        [[  5.,   6.],
         [  7.,   8.]],
 
        [[  7.,   8.],
         [  9.,  10.]],
 
        [[  9.,  10.],
         [ 11.,  12.]],
 
        [[ 11.,  12.],
         [ 13.,  14.]]]), array([[  5.,   6.],
        [  7.,   8.],
        [  9.,  10.],
        [ 11.,  12.],
        [ 13.,  14.],
        [ 15.,  16.]]))

In [8]:
def generateModelData((data1,target1), n_last_timestamps_as_features = 200, 
                      v = False, build_all_model = True, p_train_general = 1.0, 
                      stddev_factor = 1.0, returnSmoothed = False):
    
    N_total = data1.shape[0]
    N_total_test = N_total * 0.4
    
    #(data,target) = getMovementsOnly(data1, target1)
    (data,target) = (data1, target1)
    N = data.shape[0]
    
    print data.shape
    print target.shape
    
    p_train = 0.6
    X_train = data[0:(N * p_train_general * p_train),:]
    y_train = target[0:(N * p_train_general * p_train),:]
    X_test = data[((N * p_train) + n_last_timestamps_as_features):N,:]
    y_test = target[((N * p_train) + n_last_timestamps_as_features):N,:]
    
    N_total_test = X_test.shape[0]
    

    regr = linear_model.LinearRegression(normalize=False)
    
    dist_train = getDistance(y_train)
    
    #impossible threshold
    impossible_threshold = np.array([stddev_factor, stddev_factor])


    # Train the model using the training sets
    regr.fit(X_train, y_train);
    
    #training error
    dist_train = getDistance(y_train)
    
    prediction_train = regr.predict(X_train)
    
    #target y is constrained to be within in [0,1]
    prediction_train[prediction_train < 0.0] = 0
    prediction_train[prediction_train > 1.0] = 1
    
    diff_train = np.abs(prediction_train - y_train)
    diff_x_train = diff_train[:,0]
    diff_y_train = diff_train[:,1]
    
    train_error_mse = (np.sum((np.power(diff_x_train, 2) + np.power(diff_y_train, 2))) / diff_x_train.shape[0])
    train_error_dist = (np.sum((np.power(diff_x_train, 2) + np.power(diff_y_train, 2))) / dist_train)
    
    (smoothed_time_error_train, smoothed_distance_error_train, N_smoothed_train) = \
    get_smoothed_result(X_train,y_train, diff_train, impossible_threshold, v = False) 
   

    #test error
    dist_test = getDistance(y_test)
    
    prediction_test = regr.predict(X_test)
    
    #target y is constrained to be within in [0,1]
    prediction_test[prediction_test < 0.0] = 0
    prediction_test[prediction_test > 1.0] = 1
    
    diff_test = np.abs(prediction_test - y_test)        
    diff_x_test = diff_test[:,0]
    diff_y_test = diff_test[:,1]
    
    test_error_mse = (np.sum(np.power(diff_x_test, 2) + np.power(diff_y_test, 2)) / diff_x_test.shape[0])
    test_error_dist = (np.sum(np.power(diff_x_test, 2) + np.power(diff_y_test, 2)) / dist_test)
    
    (smoothed_time_error_test, smoothed_distance_error_test, N_smoothed) = \
    get_smoothed_result(X_test,y_test, diff_test, impossible_threshold, v = v) 
    
    if (v):
        
        print("threshold: " + str(impossible_threshold))
           
        #Prediction of x-coordinate of the cursor over time
        plt.plot(y_test[:,0])
        plt.plot(prediction_test[:,0])
        plt.ylabel('relative postion - x coordinate of the cursor')
        plt.title("regression - x")
        plt.show()
        
        #Prediction of y-coordinate of the cursor over time
        plt.plot(y_test[:,1])
        plt.plot(prediction_test[:,1])
        plt.ylabel('relative postion - y coordinate of the cursor')
        plt.title("regression - y")
        plt.show()
        
        print("test - MSE: %.10f" % test_error_mse)
        print("test - MSE - dist: %.10f" % test_error_dist)
        
        print ("test - smoothed MSE: " + str(smoothed_time_error_test))
        print ("test - smoothed MSE per dist: " + str(smoothed_distance_error_test))
        
        #Test error over time
        threshold_line = np.ones(diff_x_test.shape) * impossible_threshold[0]        
        plt.plot(diff_x_test)
        plt.plot(threshold_line)
        plt.ylabel('Test error')
        plt.title("ErrorPerTestTime - x")
        plt.show()
        threshold_line = np.ones(diff_y_test.shape) * impossible_threshold[1]        
        plt.plot(diff_y_test)
        plt.plot(threshold_line)
        plt.ylabel('Test error')
        plt.title("ErrorPerTestTime - y")
        plt.show()
        
        #Training error over time
        threshold_line = np.ones(diff_x_train.shape) * impossible_threshold[0]
        plt.plot(diff_x_train)
        plt.plot(threshold_line)
        plt.ylabel('Training error')
        plt.title("ErrorPerTrainTime - x")
        plt.show()
        
        threshold_line = np.ones(diff_y_train.shape) * impossible_threshold[1]
        plt.plot(diff_y_train)
        plt.plot(threshold_line)
        plt.ylabel('Training error')
        plt.title("ErrorPerTrainTime - y")
        plt.show()
        
        print("train - MSE: %.10f" % train_error_mse)
        print("train - MSE - dist: %.10f" % train_error_dist)    
        
        print ("train - smoothed MSE: " + str(smoothed_time_error_train))
        print ("train - smoothed MSE per dist: " + str(smoothed_distance_error_train))
    
        
    if (returnSmoothed):
        return((smoothed_time_error_test, smoothed_distance_error_test, float(N_smoothed) / float(N_total_test)))
    
    if (build_all_model):   
        final_regr = linear_model.LinearRegression(normalize=True)
    
        final_regr.fit(data, target);
    
        return ((final_regr, impossible_threshold))
        #return (regr, two_sigma)
    else:
        return test_error_mse

In [9]:
def generateModel(dataFile, n_last_timestamps_as_features = 200, v = False, stddev_factor = 0.0):
    (data,target) = extractFeatures(dataFile, n_last_timestamps_as_features)
    
    return generateModelData((data,target), n_last_timestamps_as_features = n_last_timestamps_as_features, 
                             v = v, stddev_factor = stddev_factor)

In [10]:
def getMovementsOnly(data, target):
    moveOnlyData = []
    moveOnlyTarget = []
    
    first = True
    
    for l in range(data.shape[0]):
        if (first):
            
            #print data[l]
            
            moveOnlyData = [data[l]]
            moveOnlyTarget = [target[l]]
            first = False
        else:
            
            #print ("new: " + str(data[l]))
            #print ("old: " + str(data[l]))
            
            if (np.any(data[l] != data[l-1]) or np.any(target[l] != target[l-1])):
                moveOnlyData = np.concatenate((moveOnlyData, [data[l]]), axis=0)
                moveOnlyTarget = np.concatenate((moveOnlyTarget, [target[l]]), axis=0)

    return (np.matrix(moveOnlyData), np.matrix(moveOnlyTarget))
            

In [11]:
def euclidean (p1, p2):
    return np.sqrt(np.sum(np.power(p1 - p2, 2)))

def getDistance(target):
    distance = 0.0
    for t in np.arange(1, target.shape[0], 1):
        distance += euclidean(target[t,:], target[(t-1),:])
    
    return distance    

def getSmoothedDistance(target, isOverThreshold):
    
    distance = 0.0
    for t in np.arange(1, target.shape[0], 1):
        last_point_x = target[t-1,0]
        last_point_y = target[t-1,1]
        if (isOverThreshold[t] == False):
            distance += euclidean(target[t,:], np.array([last_point_x,last_point_y])) 
    
    return distance

In [12]:
def get_smoothed_result(newData, newTarget, diff, threshold, v = False):
    
    diff_smoothed = np.copy(diff)
    
    N_smoothed = newTarget.shape[0]
    
    is_over_threshold = np.logical_or((diff_smoothed[:,0] > threshold[0]),(diff_smoothed[:,1] > threshold[1]))
    
    newdist = getSmoothedDistance(newTarget, is_over_threshold)
    
    for l in range(diff_smoothed.shape[0]):
        if (is_over_threshold[l]):
            diff_smoothed[l,0] = 0
            diff_smoothed[l,1] = 0
            N_smoothed -= 1
            
    if (v):
        plt.plot(diff_smoothed[:,0])
        plt.ylabel('prediction x error -smoothed')
        plt.show()

        plt.plot(diff_smoothed[:,1])
        plt.ylabel('prediction y error -smoothed')
        plt.show()
    
    time_error = 0.0
    if (N_smoothed > 0.0):
        time_error = np.sum(np.power(diff_smoothed,2)) / N_smoothed
    
    distance_error = 0.0
    if (newdist > 0.0):
        distance_error = np.sum(np.power(diff_smoothed,2)) / newdist
    
    return ((time_error,distance_error, N_smoothed))

In [13]:
def showStats(file, m, threshold, n_last_timestamps_as_features = 200, normalize = True, v = True):
    (fileX1, fileY1) = extractFeatures(file, 
                                     n_last_timestamps_as_features = n_last_timestamps_as_features, 
                                     normalize = normalize) 
    
    return showStatsData((fileX1, fileY1), m, threshold, n_last_timestamps_as_features = n_last_timestamps_as_features, v = v)

In [14]:
t = 1000000.0
    
for i in range(1000001):
    t += 0.000001

t -= 1000000.0

print t

1.0000086145


In [15]:
def showStatsData((fileX, fileY), m, threshold, n_last_timestamps_as_features = 200, v = True):
    
    (newData, newTarget) = getMovementsOnly(fileX, fileY)   
    
    if newData.shape[0] <= 1: return(0.0, 0.0, 0, 1)
    
    #distance
    dist = getDistance(newTarget)
    
    prediction_sample = m.predict(newData)
    
    #target y is constrained to be within in [0,1]
    prediction_sample[prediction_sample < 0.0] = 0
    prediction_sample[prediction_sample > 1.0] = 1

    ErrorPerTestTime = np.sum(np.absolute(prediction_sample - newTarget),1);
    
    threshold_line = np.ones(ErrorPerTestTime.shape) * (threshold[0] + threshold[1])
    
    if (v):
        #error of sample
        plt.plot(ErrorPerTestTime)
        plt.plot(threshold_line)
        plt.ylabel('prediction error')
        plt.show()

        #Prediction of x-coordinate of the cursor over time
        plt.plot(prediction_sample[:,0])
        plt.plot(newData[:,0])
        plt.ylabel('relative x-coordinate')
        plt.title("Regression for x-coordinate")    
        plt.show()

        #Prediction of y-coordinate of the cursor over time
        plt.plot(prediction_sample[:,1])
        plt.plot(newData[:,1])
        plt.ylabel('relative y-coordinate')
        plt.title("Regression for y-coordinate")
        plt.show()

    diff = np.absolute(prediction_sample - newTarget)
    
    N = newTarget.shape[0]
    
    if (v):
        print ("MSE per 0.03s: " + str(np.sum(np.power(diff,2)) / N))
        print ("MSE per pixel move: " + str(np.sum(np.power(diff,2)) / dist))
        print ("number of records: " + str(N))

        print ("\nmax - |x+y|: " + str(np.max(diff)))
        print ("max - squared: " + str(np.max(np.power(diff,2))))
    
    (time_error, distance_error, N_smoothed) = get_smoothed_result(newData, newTarget, diff, threshold)    
    
    if (v):
        print ("smoothed MSE: " + str(time_error))
        print ("smoothed MSE per dist: " + str(distance_error))
    
    if (time_error > theta): 
        if (v):
            print ("This is not Felix - You are unauthorized!")
        return (time_error, distance_error, N_smoothed, 0)
    else:
        if (v):
            print ("This is Felix - Welcome to this awesome computer!")  
        return (time_error, distance_error, N_smoothed, 1)

In [12]:
p_train = 0.6
p_valid = 0.2
#p_test = 0.2

rr_steps = 100

(data,target) = extractFeatures("/home/felix/mousetracking/BehavioralAuthentication/src/main/java/features3.txt", \
                                rr_steps)

print(data.shape)

N = data.shape[0]

train_dataset = data[0:(N * p_train),:,:]
train_labels = target[0:(N * p_train),:]
valid_dataset = data[((N * p_train) + rr_steps):((N * (p_train + p_valid))),:,:]
valid_labels = target[((N * p_train) + rr_steps):((N * (p_train + p_valid))),:]
test_dataset = data[((N * (p_train + p_valid)) + rr_steps):N,:,:]
test_labels = target[((N * (p_train + p_valid)) + rr_steps):N,:]

#(train_dataset,train_labels) = getMovementsOnly(train_dataset, train_labels)

print(train_dataset.shape)
print(train_labels.shape)

num_features = train_dataset.shape[1]
num_labels = train_labels.shape[1]

186051
original shape: (185950, 100, 2)
(185950, 100, 2)
(111570, 100, 2)
(111570, 2)


In [54]:
def accuracy(predictions, labels):
  
  return np.sum((np.square(np.matrix(predictions) - np.matrix(labels))).mean(axis=0))

In [14]:
import pandas as pd  
from random import random

from keras.models import Sequential  
from keras.layers import Dense, Dropout, Activation  
from keras.layers.recurrent import LSTM

in_neurons = 2  
out_neurons = 2  
hidden_neurons = 20

model = Sequential()  
model.add(LSTM(output_dim=hidden_neurons, input_dim=in_neurons, return_sequences=False)) 
#model.add(Dropout(0.5))
model.add(Dense(output_dim=out_neurons, input_dim=hidden_neurons))
model.add(Activation("softmax"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop") 

#(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data
model.fit(train_dataset, train_labels, batch_size=450, nb_epoch=10, validation_split=0.05)  

predicted = model.predict(test_dataset)  
rmse = np.sqrt(((predicted - test_labels) ** 2).mean(axis=0))

print rmse

Train on 105991 samples, validate on 5579 samples
Epoch 1/10
105991/105991 [==============================] - 95s - loss: 0.0536 - val_loss: 0.0258
Epoch 2/10
105991/105991 [==============================] - 107s - loss: 0.0512 - val_loss: 0.0255
Epoch 3/10
105991/105991 [==============================] - 102s - loss: 0.0512 - val_loss: 0.0254
Epoch 4/10
105991/105991 [==============================] - 106s - loss: 0.0511 - val_loss: 0.0253
Epoch 5/10
105991/105991 [==============================] - 132s - loss: 0.0511 - val_loss: 0.0253
Epoch 6/10
105991/105991 [==============================] - 116s - loss: 0.0511 - val_loss: 0.0253
Epoch 7/10
105991/105991 [==============================] - 105s - loss: 0.0511 - val_loss: 0.0252
Epoch 8/10
105991/105991 [==============================] - 106s - loss: 0.0511 - val_loss: 0.0252
Epoch 9/10
105991/105991 [==============================] - 105s - loss: 0.0510 - val_loss: 0.0252
Epoch 10/10
105991/105991 [==============================] -

In [15]:
print rmse

[ 0.1528915   0.15456401]


In [22]:
#with dropout
in_neurons = 2  
out_neurons = 2  
hidden_neurons = 20

model = Sequential()  
model.add(LSTM(output_dim=hidden_neurons, input_dim=in_neurons, return_sequences=False)) 
model.add(Dropout(0.5))
model.add(Dense(output_dim=out_neurons, input_dim=hidden_neurons))
model.add(Activation("softmax"))  
model.compile(loss="mean_squared_error", optimizer="rmsprop") 

#(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data
model.fit(train_dataset, train_labels, batch_size=450, nb_epoch=10, validation_split=0.05)  

predicted = model.predict(test_dataset)  
rmse = np.sqrt(((predicted - test_labels) ** 2).mean(axis=0))

print rmse

Train on 105991 samples, validate on 5579 samples
Epoch 1/10
105991/105991 [==============================] - 73s - loss: 0.0597 - val_loss: 0.0264
Epoch 2/10
105991/105991 [==============================] - 98s - loss: 0.0537 - val_loss: 0.0258
Epoch 3/10
105991/105991 [==============================] - 100s - loss: 0.0531 - val_loss: 0.0257
Epoch 4/10
105991/105991 [==============================] - 97s - loss: 0.0528 - val_loss: 0.0255
Epoch 5/10
105991/105991 [==============================] - 104s - loss: 0.0526 - val_loss: 0.0254
Epoch 6/10
105991/105991 [==============================] - 98s - loss: 0.0525 - val_loss: 0.0254
Epoch 7/10
105991/105991 [==============================] - 93s - loss: 0.0524 - val_loss: 0.0253
Epoch 8/10
105991/105991 [==============================] - 102s - loss: 0.0524 - val_loss: 0.0253
Epoch 9/10
105991/105991 [==============================] - 95s - loss: 0.0523 - val_loss: 0.0253
Epoch 10/10
105991/105991 [==============================] - 92s 

In [38]:
in_neurons = 2  
out_neurons = 2  
hidden_neurons = 20
batch_size = 500
time_steps = 100
training_size = 110000 # needs to be divisable by batch_size
test_size = 37000 # needs to be divisable by batch_size

model3 = Sequential()  
model3.add(LSTM(output_dim=hidden_neurons, batch_input_shape=(batch_size, time_steps, in_neurons), stateful=True, return_sequences=False)) 
model3.add(Dense(output_dim=out_neurons, activation='softmax'))

model3.compile(loss="mean_squared_error", optimizer="rmsprop") 

#(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data
model3.fit(train_dataset[0:training_size], train_labels[0:training_size], batch_size=batch_size, nb_epoch=10, validation_split=0.1, show_accuracy=True)  

predicted = model3.predict(test_dataset[0:test_size], batch_size=batch_size)  
rmse = np.sqrt(((predicted - test_labels[0:test_size]) ** 2).mean(axis=0))

print rmse

Train on 99000 samples, validate on 11000 samples
Epoch 1/10
99000/99000 [==============================] - 70s - loss: 0.0569 - acc: 0.8880 - val_loss: 0.0138 - val_acc: 0.9969
Epoch 2/10
99000/99000 [==============================] - 70s - loss: 0.0544 - acc: 0.9565 - val_loss: 0.0124 - val_acc: 0.9973
Epoch 3/10
99000/99000 [==============================] - 74s - loss: 0.0543 - acc: 0.9777 - val_loss: 0.0123 - val_acc: 0.9980
Epoch 4/10
99000/99000 [==============================] - 77s - loss: 0.0543 - acc: 0.9879 - val_loss: 0.0122 - val_acc: 0.9982
Epoch 5/10
99000/99000 [==============================] - 81s - loss: 0.0542 - acc: 0.9901 - val_loss: 0.0122 - val_acc: 0.9983
Epoch 6/10
99000/99000 [==============================] - 81s - loss: 0.0542 - acc: 0.9918 - val_loss: 0.0122 - val_acc: 0.9984
Epoch 7/10
99000/99000 [==============================] - 83s - loss: 0.0542 - acc: 0.9946 - val_loss: 0.0122 - val_acc: 0.9985
Epoch 8/10
99000/99000 [==============================

In [36]:
predicted = model3.predict(test_dataset[0:500])  
#rmse = np.sqrt(((predicted - test_labels[0:500]) ** 2).mean(axis=0))

ValueError: Shape mismatch: x has 500 rows but z has 128 rows
Apply node that caused the error: Gemm{inplace}(Dot22.0, TensorConstant{0.20000000298}, <TensorType(float32, matrix)>, <TensorType(float32, matrix)>, TensorConstant{0.20000000298})
Toposort index: 10
Inputs types: [TensorType(float32, matrix), TensorType(float32, scalar), TensorType(float32, matrix), TensorType(float32, matrix), TensorType(float32, scalar)]
Inputs shapes: [(128, 20), (), (500, 20), (20, 20), ()]
Inputs strides: [(80, 4), (), (80, 4), (80, 4), ()]
Inputs values: ['not shown', array(0.20000000298023224, dtype=float32), 'not shown', 'not shown', array(0.20000000298023224, dtype=float32)]
Outputs clients: [[Elemwise{Composite{(clip((i0 + i1 + i2), i3, i4) * tanh(i5))}}(TensorConstant{(1, 1) of 0.5}, Elemwise{mul,no_inplace}.0, Gemm{inplace}.0, TensorConstant{(1, 1) of 0}, TensorConstant{(1, 1) of 1}, Elemwise{Composite{((clip((i0 + i1 + i2), i3, i4) * i5) + (clip((i6 + i7 + i8), i3, i4) * tanh((i9 + i10))))}}.0)

In [32]:
test_dataset.shape

(37090, 100, 2)

In [21]:
in_neurons = 2  
out_neurons = 2  
h1 = 300
h2 = 500
h3 = 200

model2 = Sequential()  

model2.add(LSTM(output_dim=h1, input_dim=in_neurons, return_sequences=True))  
model2.add(LSTM(output_dim=h2, input_dim=h1, return_sequences=True))  
model2.add(Dropout(0.5))  
model2.add(LSTM(output_dim=h3, input_dim=h2, return_sequences=False))  
model2.add(Dropout(0.5))  
model2.add(Dense(output_dim=out_neurons, input_dim=h3))
model2.add(Activation("softmax"))  
model2.compile(loss="mean_squared_error", optimizer="rmsprop") 

#(X_train, y_train), (X_test, y_test) = train_test_split(data)  # retrieve data
model2.fit(train_dataset, train_labels, batch_size=450, nb_epoch=10, validation_split=0.05)  

predicted = model2.predict(test_dataset)  
rmse = np.sqrt(((predicted - test_labels) ** 2).mean(axis=0))


Train on 105991 samples, validate on 5579 samples
Epoch 1/10
105991/105991 [==============================] - 14758s - loss: 0.0591 - val_loss: 0.0260
Epoch 2/10
105991/105991 [==============================] - 14351s - loss: 0.0520 - val_loss: 0.0259
Epoch 3/10
 24300/105991 [=====>........................] - ETA: 10753s - loss: 0.0516

KeyboardInterrupt: 